In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
raw_examples = [
    {"text":"Have you used the new version of my model?","spans":[{"start":36,"end":41,"token_start":8,"token_end":8,"label":"SKILL"}],"_input_hash":1798863398,"_task_hash":1273875979,"tokens":[{"text":"Have","start":0,"end":4,"id":0},{"text":"you","start":5,"end":8,"id":1},{"text":"used","start":9,"end":13,"id":2},{"text":"the","start":14,"end":17,"id":3},{"text":"new","start":18,"end":21,"id":4},{"text":"version","start":22,"end":29,"id":5},{"text":"of","start":30,"end":32,"id":6},{"text":"my","start":33,"end":35,"id":7},{"text":"model","start":36,"end":41,"id":8},{"text":"?","start":41,"end":42,"id":9}],"_session_id":None,"_view_id":"ner_manual","answer":"accept"},
    {"text":"I'd like to work as an actor or model if possible.","spans":[{"text":"actor","start":23,"end":28,"token_start":7,"token_end":7,"label":"JOB_ROLE"},{"text":"model","start":32,"end":37,"token_start":9,"token_end":9,"label":"JOB_ROLE"}],"_input_hash":895552771,"_task_hash":-936257555,"tokens":[{"text":"I","start":0,"end":1,"id":0},{"text":"'d","start":1,"end":3,"id":1},{"text":"like","start":4,"end":8,"id":2},{"text":"to","start":9,"end":11,"id":3},{"text":"work","start":12,"end":16,"id":4},{"text":"as","start":17,"end":19,"id":5},{"text":"an","start":20,"end":22,"id":6},{"text":"actor","start":23,"end":28,"id":7},{"text":"or","start":29,"end":31,"id":8},{"text":"model","start":32,"end":37,"id":9},{"text":"if","start":38,"end":40,"id":10},{"text":"possible","start":41,"end":49,"id":11},{"text":".","start":49,"end":50,"id":12}],"_session_id":None,"_view_id":"ner_manual","answer":"accept"},
    {"text":"We are looking for a Software Development Engineer who has solid coding skills, a strong machine learning background, and is passionate about developing new AI products.","meta":{"id":"599073","category":"Engineering","subCategory":"Data & Applied Sciences"},"_input_hash":-275722772,"_task_hash":868828486,"_session_id":None,"_view_id":"ner_manual","answer":"accept","dataset":1,"tokens":[{"text":"We","start":0,"end":2,"id":0},{"text":"are","start":3,"end":6,"id":1},{"text":"looking","start":7,"end":14,"id":2},{"text":"for","start":15,"end":18,"id":3},{"text":"a","start":19,"end":20,"id":4},{"text":"Software","start":21,"end":29,"id":5},{"text":"Development","start":30,"end":41,"id":6},{"text":"Engineer","start":42,"end":50,"id":7},{"text":"who","start":51,"end":54,"id":8},{"text":"has","start":55,"end":58,"id":9},{"text":"solid","start":59,"end":64,"id":10},{"text":"coding","start":65,"end":71,"id":11},{"text":"skills","start":72,"end":78,"id":12},{"text":",","start":78,"end":79,"id":13},{"text":"a","start":80,"end":81,"id":14},{"text":"strong","start":82,"end":88,"id":15},{"text":"machine","start":89,"end":96,"id":16},{"text":"learning","start":97,"end":105,"id":17},{"text":"background","start":106,"end":116,"id":18},{"text":",","start":116,"end":117,"id":19},{"text":"and","start":118,"end":121,"id":20},{"text":"is","start":122,"end":124,"id":21},{"text":"passionate","start":125,"end":135,"id":22},{"text":"about","start":136,"end":141,"id":23},{"text":"developing","start":142,"end":152,"id":24},{"text":"new","start":153,"end":156,"id":25},{"text":"AI","start":157,"end":159,"id":26},{"text":"products","start":160,"end":168,"id":27},{"text":".","start":168,"end":169,"id":28}],"spans":[{"start":21,"end":50,"token_start":5,"token_end":7,"label":"SKILL"},{"start":65,"end":71,"token_start":11,"token_end":11,"label":"SKILL"},{"start":89,"end":105,"token_start":16,"token_end":17,"label":"SKILL"},{"start":142,"end":152,"token_start":24,"token_end":24,"label":"SKILL"},{"start":157,"end":159,"token_start":26,"token_end":26,"label":"SKILL"}]},
    {"text":"Responsibilities As a SOFTWARE DEVELOPMENT ENGINEER II you will work / collaborate with other talented engineers to build features and technologies that will affect millions of your fellow developers in the community.","meta":{"id":"598569","category":"Engineering","subCategory":"Software Engineering"},"_input_hash":-930202924,"_task_hash":547318084,"_session_id":None,"_view_id":"ner_manual","answer":"accept","dataset":1,"tokens":[{"text":"Responsibilities","start":0,"end":16,"id":0},{"text":"As","start":17,"end":19,"id":1},{"text":"a","start":20,"end":21,"id":2},{"text":"SOFTWARE","start":22,"end":30,"id":3},{"text":"DEVELOPMENT","start":31,"end":42,"id":4},{"text":"ENGINEER","start":43,"end":51,"id":5},{"text":"II","start":52,"end":54,"id":6},{"text":"you","start":55,"end":58,"id":7},{"text":"will","start":59,"end":63,"id":8},{"text":"work","start":64,"end":68,"id":9},{"text":"/","start":69,"end":70,"id":10},{"text":"collaborate","start":71,"end":82,"id":11},{"text":"with","start":83,"end":87,"id":12},{"text":"other","start":88,"end":93,"id":13},{"text":"talented","start":94,"end":102,"id":14},{"text":"engineers","start":103,"end":112,"id":15},{"text":"to","start":113,"end":115,"id":16},{"text":"build","start":116,"end":121,"id":17},{"text":"features","start":122,"end":130,"id":18},{"text":"and","start":131,"end":134,"id":19},{"text":"technologies","start":135,"end":147,"id":20},{"text":"that","start":148,"end":152,"id":21},{"text":"will","start":153,"end":157,"id":22},{"text":"affect","start":158,"end":164,"id":23},{"text":"millions","start":165,"end":173,"id":24},{"text":"of","start":174,"end":176,"id":25},{"text":"your","start":177,"end":181,"id":26},{"text":"fellow","start":182,"end":188,"id":27},{"text":"developers","start":189,"end":199,"id":28},{"text":"in","start":200,"end":202,"id":29},{"text":"the","start":203,"end":206,"id":30},{"text":"community","start":207,"end":216,"id":31},{"text":".","start":216,"end":217,"id":32}],"spans":[{"start":22,"end":51,"token_start":3,"token_end":5,"label":"JOB_ROLE"},{"start":71,"end":82,"token_start":11,"token_end":11,"label":"SKILL"},{"start":103,"end":112,"token_start":15,"token_end":15,"label":"JOB_ROLE"},{"start":135,"end":147,"token_start":20,"token_end":20,"label":"SKILL"},{"start":189,"end":199,"token_start":28,"token_end":28,"label":"JOB_ROLE"}]}
]

for e in raw_examples:
#     del e['tokens']
    del e['_input_hash']
    del e['_task_hash']
#     for s in e['spans']:
#         del s['token_start']
#         del s['token_end']
        
    del_keys = [k for k in e.keys() if k not in {'text', 'spans', 'tokens'}]
    
    for k in del_keys:
        del e[k]

import json
for e in raw_examples:
    print(json.dumps(e) + ",")

In [6]:
import copy
from pprint import pprint
from pathlib import Path
from typing import Any, Dict, List, Set, Tuple
import spacy
import srsly
# import recon
from recon.corpus import Corpus
from recon.operations.corrections import fix_annotations
from recon.dataset import Dataset
from recon.loaders import read_jsonl
from recon.types import Correction, Example, PredictionError, HardestExample, NERStats, EntityCoverageStats, EntityCoverage, Transformation, TransformationType, OperationState
from recon.stats import (
    get_ner_stats, get_entity_coverage, get_sorted_type_counts, get_probs_from_counts, entropy,
    calculate_entity_coverage_entropy, calculate_label_balance_entropy, calculate_label_distribution_similarity,
    detect_outliers
)
import recon.operations.tokenization as tokenization
from recon.insights import get_ents_by_label, get_label_disparities, top_prediction_errors, top_label_disparities, get_hardest_examples
from recon.recognizer import SpacyEntityRecognizer
from recon.operations import registry
from recon.store import ExampleStore

In [7]:
registry.operations.get_all()

{'recon.v1.augment.ent_label_sub': <recon.operations.core.Operation at 0x117af1c00>,
 'recon.v1.rename_labels': <recon.operations.core.Operation at 0x127942020>,
 'recon.v1.fix_annotations': <recon.operations.core.Operation at 0x127942110>,
 'recon.v1.strip_annotations': <recon.operations.core.Operation at 0x127942170>,
 'recon.v1.split_sentences': <recon.operations.core.Operation at 0x1279421d0>,
 'recon.v1.fix_tokenization_and_spacing': <recon.operations.core.Operation at 0x127942320>,
 'recon.v1.add_tokens': <recon.operations.core.Operation at 0x127942380>,
 'recon.v1.upcase_labels': <recon.operations.core.Operation at 0x1279425c0>,
 'recon.v1.filter_overlaps': <recon.operations.core.Operation at 0x127942650>}

In [9]:
fix_tokenization_and_spacing = registry.operations.get("recon.v1.fix_tokenization_and_spacing")
add_tokens = registry.operations.get("recon.v1.add_tokens")
rename_labels = registry.operations.get("recon.v1.rename_labels")
fix_annotations = registry.operations.get("recon.v1.fix_annotations")
upcase_labels = registry.operations.get("recon.v1.upcase_labels")
filter_overlaps = registry.operations.get("recon.v1.filter_overlaps")

In [10]:
corpus = Corpus.from_disk("./data/skills")

In [11]:
corpus._train.operations

[]

In [12]:
print(get_ner_stats(corpus._train.data, serialize=True))

{
    "n_examples":106,
    "n_examples_no_entities":29,
    "n_annotations":243,
    "n_annotations_per_type":{
        "SKILL":197,
        "PRODUCT":33,
        "JOB_ROLE":10,
        "skill":2,
        "product":1
    },
    "examples_with_type":null
}


In [14]:
corpus.apply_("recon.v1.fix_tokenization_and_spacing")

=> Applying operation 'recon.v1.fix_tokenization_and_spacing' to dataset 'train'


  0%|                                                                                                             | 0/106 [00:00<?, ?it/s]

ℹ      => Running preprocessor recon.v1.spacy



100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:00<00:00, 116.05it/s]


✔ Completed operation 'recon.v1.fix_tokenization_and_spacing'
=> Applying operation 'recon.v1.fix_tokenization_and_spacing' to dataset 'dev'


...

✔ Completed operation 'recon.v1.fix_tokenization_and_spacing'
=> Applying operation 'recon.v1.fix_tokenization_and_spacing' to dataset 'test'


  0%|                                                                                                              | 0/96 [00:00<?, ?it/s]

ℹ      => Running preprocessor recon.v1.spacy



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:01<00:00, 95.85it/s]

✔ Completed operation 'recon.v1.fix_tokenization_and_spacing'


In [15]:
corpus._train.operations

[OperationState(name='recon.v1.fix_tokenization_and_spacing', batch=False, args=(), kwargs={}, status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2022, 4, 10, 19, 47, 25, 440013), examples_added=0, examples_removed=0, examples_changed=0, transformations=[])]

In [18]:
corpus.apply_("recon.v1.upcase_labels")

=> Applying operation 'recon.v1.upcase_labels' to dataset 'train'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:01<00:00, 95.55it/s]


✔ Completed operation 'recon.v1.upcase_labels'
=> Applying operation 'recon.v1.upcase_labels' to dataset 'dev'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [00:01<00:00, 95.08it/s]


✔ Completed operation 'recon.v1.upcase_labels'
=> Applying operation 'recon.v1.upcase_labels' to dataset 'test'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 99.08it/s]

✔ Completed operation 'recon.v1.upcase_labels'


In [19]:
print(corpus.apply(get_ner_stats, serialize=True).train)

{
    "n_examples":106,
    "n_examples_no_entities":29,
    "n_annotations":243,
    "n_annotations_per_type":{
        "SKILL":199,
        "PRODUCT":34,
        "JOB_ROLE":10
    },
    "examples_with_type":null
}


In [20]:
corpus._train.operations

[OperationState(name='recon.v1.fix_tokenization_and_spacing', batch=False, args=(), kwargs={}, status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2022, 4, 10, 19, 47, 25, 440013), examples_added=0, examples_removed=0, examples_changed=0, transformations=[]),
 OperationState(name='recon.v1.upcase_labels', batch=False, args=(), kwargs={}, status=<OperationStatus.COMPLETED: 'COMPLETED'>, ts=datetime.datetime(2022, 4, 10, 19, 47, 25, 440013), examples_added=0, examples_removed=0, examples_changed=3, transformations=[Transformation(prev_example=183104467179452678, example=594969270528231551, type=<TransformationType.EXAMPLE_CHANGED: 'EXAMPLE_CHANGED'>), Transformation(prev_example=3933994642316611414, example=4880013973796351025, type=<TransformationType.EXAMPLE_CHANGED: 'EXAMPLE_CHANGED'>), Transformation(prev_example=1365997553723636798, example=6743399603562727171, type=<TransformationType.EXAMPLE_CHANGED: 'EXAMPLE_CHANGED'>)])]

In [21]:
corpus.to_disk('./fixed_data/skills', force=True)

In [6]:
corpus2 = Corpus.from_disk('./fixed_data/skills')

In [23]:
len(corpus._train.operations), len(corpus._dev.operations), len(corpus._test.operations)

(5, 5, 5)

In [37]:
corpus2.apply(get_label_disparities, "SKILL", "JOB_ROLE")

{'train': {'model'}, 'dev': set(), 'test': set(), 'all': {'model'}}

In [19]:
import numpy as np
 
# Create a 5 x 5 ndarray with consecutive integers from 1 to 25 (inclusive).
# Afterwards use Boolean indexing to pick out only the odd numbers in the array
 
# Create a 5 x 5 ndarray with consecutive integers from 1 to 25 (inclusive).
X = np.linspace(1,25,25).reshape(5,5)
print(X)
 
# Use Boolean indexing to pick out only the odd numbers in the array
X * X % 2 == 1
# Y = X[]
# print(Y)

[[ 1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15.]
 [16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25.]]


array([[ True, False,  True, False,  True],
       [False,  True, False,  True, False],
       [ True, False,  True, False,  True],
       [False,  True, False,  True, False],
       [ True, False,  True, False,  True]])

In [25]:
def corrections_from_dict(corrections_dict: Dict[str, Any]):
    corrections: List[Correction] = []
    for key, val in corrections_dict.items():
        if isinstance(val, str):
            from_label = "ANY"
            to_label = val
        elif isinstance(val, tuple):
            from_label = val[0]
            to_label = val[1]
        else:
            raise ValueError("Cannot parse corrections dict. Value must be either a str of the label " +
                             "to change the annotation to (TO_LABEL) or a tuple of (FROM_LABEL, TO_LABEL)")
        corrections.append(Correction(
            annotation=key,
            from_label=from_label,
            to_label=to_label
        ))
    return corrections

corrections_from_dict({
    "model": ("PRODUCT", "SKILL"),
})

[Correction(annotation='model', from_label='PRODUCT', to_label='SKILL')]

In [10]:
corrections = [
    Correction(text="")
]
corpus2.apply_(fix_annotations, {"model": "SKILL"})

NameError: name 'corpus2' is not defined

In [41]:
[op.name for op in corpus2._train.operations]

['fix_tokenization_and_spacing', 'add_tokens', 'fix_annotations']

In [40]:
corpus2.to_disk('./fixed_data/skills_2', force=True)

In [ ]:
large_corpus.apply_(fix_tokenization_and_spacing)

In [ ]:
large_corpus.apply_(fix_tokenization_and_spacing)

In [ ]:
large_corpus.apply_(add_tokens)

In [ ]:
new_data = tokenization.fix_tokenization_and_spacing(prev_data)

from timeit import default_timer as timer
print("RUNNING OPERATION\n")

start = timer()

initial_len = len(prev_data)
new_data_len = len(new_data)

def get_examples_and_texts(data: List[Example]) -> Tuple[Dict[int, int], Dict[str, int]]:
    examples: Dict[int, int] = {}
    texts: Dict[str, int] = {}

    for i, e in enumerate(data):
        examples[hash(e)] = i
        texts[e.text_hash()] = i

    return examples, texts

prev_example_hashes, prev_texts = get_examples_and_texts(prev_data)
new_example_hashes, new_texts = get_examples_and_texts(new_data)

prev_diff = set(prev_example_hashes).difference(set(new_example_hashes))
next_diff = set(new_example_hashes).difference(set(prev_example_hashes))

prev_examples_to_hashes = {prev_example_hashes[pd]: pd for pd in prev_diff}
next_examples_to_hashes = {new_example_hashes[nd]: nd for nd in next_diff}

examples_added = max(len(next_diff) - len(prev_diff), 0)
examples_removed = max(len(prev_diff) - len(next_diff), 0)







seen = set()
transformations = set()

for pd in prev_diff:
    if pd in new_example_hashes:
        prev_example_index = prev_example_hashes[pd]
        new_example_hashes[pd] = prev_example_index
    prev_example_index = prev_example_hashes[pd]
    if prev_example_index in next_examples_to_hashes:
        new_example_hash = next_examples_to_hashes[prev_example_index]
        transformation = Transformation(prev_example=pd, example=new_example_hash, type=TransformationType.EXAMPLE_CHANGED)
    else:
        transformation = Transformation(prev_example=pd, example=-1, type=TransformationType.EXAMPLE_REMOVED)
    
    if pd not in seen and new_example_hash not in seen:
        transformations.add(transformation)
        seen.add(pd)
        seen.add(new_example_hash)

# seen = set()

for nd in next_diff:
    next_example_index = new_example_hashes[nd]
    if next_example_index in prev_examples_to_hashes:
        prev_example_hash = prev_examples_to_hashes[next_example_index]
        transformation = Transformation(prev_example=prev_example_hash, example=nd, type=TransformationType.EXAMPLE_CHANGED)
    else:
        transformation = Transformation(prev_example=-1, example=nd, type=TransformationType.EXAMPLE_ADDED)
        
    if nd not in seen and prev_example_hash not in seen:
        transformations.add(transformation)
        seen.add(nd)
        seen.add(prev_example_hash)

        
print(prev_diff)
print(next_diff)
print("Examples Added: ", examples_added)
print("Examples Removed: ", examples_removed)
print("Transformations: ", len(transformations))
print("Transformations (ADDED): ", len([t for t in transformations if t.type == TransformationType.EXAMPLE_ADDED]))
print("Transformations (REMOVED): ", len([t for t in transformations if t.type == TransformationType.EXAMPLE_REMOVED]))
print("Transformations (CHANGED): ", len([t for t in transformations if t.type == TransformationType.EXAMPLE_CHANGED]))



mutual_diff = set(prev_example_hashes) ^ set(new_example_hashes)
end = timer()
print("Total operation calculation time: ", round(end - start, 2))


# from dictdiffer import diff
# start = timer()
# hash_diff = list(diff(prev_data, new_data))
# end = timer()

# print("Total diff of examples: ", round(end - start, 2))

In [ ]:
len(new_data)

In [ ]:
len(hash_diff)

In [ ]:
print(hash_diff[0][2][0].json())
hash_diff[0][2][1].json()

In [ ]:
len([d for d in hash_diff if d[0] == "remove"])

In [ ]:
large_corpus.example_store, large_corpus._train.example_store

In [ ]:
%timeit hash(corpus._train), hash(corpus._dev), hash(corpus._test)

In [ ]:
%timeit hash(large_corpus._train), hash(large_corpus._dev), hash(large_corpus._test)

In [ ]:
%timeit hash(corpus._train)
%timeit hash(large_corpus._train)

In [ ]:
len(str(hash(corpus._train.data[0])))

In [ ]:
hash(corpus._train.data[0].spans[0])

In [ ]:
corpus._train.commit_hash

In [ ]:
print(fix_tokenization_and_spacing)

In [ ]:
corpus._train.operations = []
corpus._train.global_state = {}
corpus._dev.operations = []
corpus._dev.global_state = {}
corpus._test.operations = []
corpus._test.global_state = {}

In [ ]:
corpus.apply_(fix_tokenization_and_spacing)

In [ ]:
corpus._train.operations

In [ ]:
corpus.apply_(add_tokens)

In [ ]:
corpus.to_disk('./fixed_data/skills', force=True)

In [ ]:
import os
os.listdir('./fixed_data/skills/.recon/train')

In [33]:
from recon.operations import op_iter
from recon.preprocess import SpacyPreProcessor
from recon.registry import tokenizers
tokenizer = tokenizers.get("default")
nlp = tokenizer()
spacy_preprocessor = SpacyPreProcessor(nlp)


def test_add_tokens():
    # fmt: off
    untokenized_examples = [Example(**example) for example in [
        {"text": "Have you used the new version of my model?", "spans": [{"start": 36, "end": 41, "label": "SKILL"}]},
        {"text": "I'd like to work as an actor or model if possible.", "spans": [{"text": "actor", "start": 23, "end": 28, "label": "JOB_ROLE"}, {"text": "model", "start": 32, "end": 37, "label": "JOB_ROLE"}]},
        {"text": "We are looking for a Software Development Engineer who has solid coding skills, a strong machine learning background, and is passionate about developing new AI products.", "spans": [{"start": 21, "end": 50, "label": "SKILL"}, {"start": 65, "end": 71, "label": "SKILL"}, {"start": 89, "end": 105, "label": "SKILL"}, {"start": 142, "end": 152, "label": "SKILL"}, {"start": 157, "end": 159, "label": "SKILL"}]},
        {"text": "Responsibilities As a SOFTWARE DEVELOPMENT ENGINEER II you will work / collaborate with other talented engineers to build features and technologies that will affect millions of your fellow developers in the community.", "spans": [{"start": 22, "end": 51, "label": "JOB_ROLE"}, {"start": 71, "end": 82, "label": "SKILL"}, {"start": 103, "end": 112, "label": "JOB_ROLE"}, {"start": 135, "end": 147, "label": "SKILL"}, {"start": 189, "end": 199, "label": "JOB_ROLE"}]}
    ]]

    tokenized_examples = [Example(**example) for example in [
        {"text": "Have you used the new version of my model?", "spans": [{"start": 36, "end": 41, "token_start": 8, "token_end": 8, "label": "SKILL"}], "tokens": [{"text": "Have", "start": 0, "end": 4, "id": 0}, {"text": "you", "start": 5, "end": 8, "id": 1}, {"text": "used", "start": 9, "end": 13, "id": 2}, {"text": "the", "start": 14, "end": 17, "id": 3}, {"text": "new", "start": 18, "end": 21, "id": 4}, {"text": "version", "start": 22, "end": 29, "id": 5}, {"text": "of", "start": 30, "end": 32, "id": 6}, {"text": "my", "start": 33, "end": 35, "id": 7}, {"text": "model", "start": 36, "end": 41, "id": 8}, {"text": "?", "start": 41, "end": 42, "id": 9}]},
        {"text": "I'd like to work as an actor or model if possible.", "spans": [{"text": "actor", "start": 23, "end": 28, "token_start": 7, "token_end": 7, "label": "JOB_ROLE"}, {"text": "model", "start": 32, "end": 37, "token_start": 9, "token_end": 9, "label": "JOB_ROLE"}], "tokens": [{"text": "I", "start": 0, "end": 1, "id": 0}, {"text": "'d", "start": 1, "end": 3, "id": 1}, {"text": "like", "start": 4, "end": 8, "id": 2}, {"text": "to", "start": 9, "end": 11, "id": 3}, {"text": "work", "start": 12, "end": 16, "id": 4}, {"text": "as", "start": 17, "end": 19, "id": 5}, {"text": "an", "start": 20, "end": 22, "id": 6}, {"text": "actor", "start": 23, "end": 28, "id": 7}, {"text": "or", "start": 29, "end": 31, "id": 8}, {"text": "model", "start": 32, "end": 37, "id": 9}, {"text": "if", "start": 38, "end": 40, "id": 10}, {"text": "possible", "start": 41, "end": 49, "id": 11}, {"text": ".", "start": 49, "end": 50, "id": 12}]},
        {"text": "We are looking for a Software Development Engineer who has solid coding skills, a strong machine learning background, and is passionate about developing new AI products.", "tokens": [{"text": "We", "start": 0, "end": 2, "id": 0}, {"text": "are", "start": 3, "end": 6, "id": 1}, {"text": "looking", "start": 7, "end": 14, "id": 2}, {"text": "for", "start": 15, "end": 18, "id": 3}, {"text": "a", "start": 19, "end": 20, "id": 4}, {"text": "Software", "start": 21, "end": 29, "id": 5}, {"text": "Development", "start": 30, "end": 41, "id": 6}, {"text": "Engineer", "start": 42, "end": 50, "id": 7}, {"text": "who", "start": 51, "end": 54, "id": 8}, {"text": "has", "start": 55, "end": 58, "id": 9}, {"text": "solid", "start": 59, "end": 64, "id": 10}, {"text": "coding", "start": 65, "end": 71, "id": 11}, {"text": "skills", "start": 72, "end": 78, "id": 12}, {"text": ",", "start": 78, "end": 79, "id": 13}, {"text": "a", "start": 80, "end": 81, "id": 14}, {"text": "strong", "start": 82, "end": 88, "id": 15}, {"text": "machine", "start": 89, "end": 96, "id": 16}, {"text": "learning", "start": 97, "end": 105, "id": 17}, {"text": "background", "start": 106, "end": 116, "id": 18}, {"text": ",", "start": 116, "end": 117, "id": 19}, {"text": "and", "start": 118, "end": 121, "id": 20}, {"text": "is", "start": 122, "end": 124, "id": 21}, {"text": "passionate", "start": 125, "end": 135, "id": 22}, {"text": "about", "start": 136, "end": 141, "id": 23}, {"text": "developing", "start": 142, "end": 152, "id": 24}, {"text": "new", "start": 153, "end": 156, "id": 25}, {"text": "AI", "start": 157, "end": 159, "id": 26}, {"text": "products", "start": 160, "end": 168, "id": 27}, {"text": ".", "start": 168, "end": 169, "id": 28}], "spans": [{"start": 21, "end": 50, "token_start": 5, "token_end": 7, "label": "SKILL"}, {"start": 65, "end": 71, "token_start": 11, "token_end": 11, "label": "SKILL"}, {"start": 89, "end": 105, "token_start": 16, "token_end": 17, "label": "SKILL"}, {"start": 142, "end": 152, "token_start": 24, "token_end": 24, "label": "SKILL"}, {"start": 157, "end": 159, "token_start": 26, "token_end": 26, "label": "SKILL"}]},
        {"text": "Responsibilities As a SOFTWARE DEVELOPMENT ENGINEER II you will work / collaborate with other talented engineers to build features and technologies that will affect millions of your fellow developers in the community.", "tokens": [{"text": "Responsibilities", "start": 0, "end": 16, "id": 0}, {"text": "As", "start": 17, "end": 19, "id": 1}, {"text": "a", "start": 20, "end": 21, "id": 2}, {"text": "SOFTWARE", "start": 22, "end": 30, "id": 3}, {"text": "DEVELOPMENT", "start": 31, "end": 42, "id": 4}, {"text": "ENGINEER", "start": 43, "end": 51, "id": 5}, {"text": "II", "start": 52, "end": 54, "id": 6}, {"text": "you", "start": 55, "end": 58, "id": 7}, {"text": "will", "start": 59, "end": 63, "id": 8}, {"text": "work", "start": 64, "end": 68, "id": 9}, {"text": "/", "start": 69, "end": 70, "id": 10}, {"text": "collaborate", "start": 71, "end": 82, "id": 11}, {"text": "with", "start": 83, "end": 87, "id": 12}, {"text": "other", "start": 88, "end": 93, "id": 13}, {"text": "talented", "start": 94, "end": 102, "id": 14}, {"text": "engineers", "start": 103, "end": 112, "id": 15}, {"text": "to", "start": 113, "end": 115, "id": 16}, {"text": "build", "start": 116, "end": 121, "id": 17}, {"text": "features", "start": 122, "end": 130, "id": 18}, {"text": "and", "start": 131, "end": 134, "id": 19}, {"text": "technologies", "start": 135, "end": 147, "id": 20}, {"text": "that", "start": 148, "end": 152, "id": 21}, {"text": "will", "start": 153, "end": 157, "id": 22}, {"text": "affect", "start": 158, "end": 164, "id": 23}, {"text": "millions", "start": 165, "end": 173, "id": 24}, {"text": "of", "start": 174, "end": 176, "id": 25}, {"text": "your", "start": 177, "end": 181, "id": 26}, {"text": "fellow", "start": 182, "end": 188, "id": 27}, {"text": "developers", "start": 189, "end": 199, "id": 28}, {"text": "in", "start": 200, "end": 202, "id": 29}, {"text": "the", "start": 203, "end": 206, "id": 30}, {"text": "community", "start": 207, "end": 216, "id": 31}, {"text": ".", "start": 216, "end": 217, "id": 32}], "spans": [{"start": 22, "end": 51, "token_start": 3, "token_end": 5, "label": "JOB_ROLE"}, {"start": 71, "end": 82, "token_start": 11, "token_end": 11, "label": "SKILL"}, {"start": 103, "end": 112, "token_start": 15, "token_end": 15, "label": "JOB_ROLE"}, {"start": 135, "end": 147, "token_start": 20, "token_end": 20, "label": "SKILL"}, {"start": 189, "end": 199, "token_start": 28, "token_end": 28, "label": "JOB_ROLE"}]}
    ]]
    # fmt: on
    
    fixed_examples = []
    for orig_example_hash, example, preprocessed_outputs in op_iter(untokenized_examples, pre=[spacy_preprocessor]):
#         print(preprocessed_outputs)
        fixed_examples.append(tokenization.add_tokens(example, preprocessed_outputs=preprocessed_outputs))

    for fixed_example, tokenized_example in zip(fixed_examples, tokenized_examples):
        assert fixed_example.text == tokenized_example.text
        assert fixed_example.spans == tokenized_example.spans
        assert fixed_example.tokens == tokenized_example.tokens
        
test_add_tokens()